In [1]:
import pandas as pd 
import numpy as np
#tdqm = progress bar
from tqdm import tqdm
from datetime import datetime

from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn import utils



In [2]:
# Number of rows
nrows = 55000000
# Chunksize = 10000 
chunksize = 10000
# Vitou's path
# path = 'C:/Users/sirus/Downloads/train.csv'
# Muna's path
path = '/Users/muna/Development/DataScience/new-york-city-taxi-fare-prediction/train.csv'
# Columns to read from the data
# Todo :split date into meaningful data
# cols = ['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count','fare_amount']
cols = ['pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count','fare_amount']

#list to hold the batch dataframe
df_list = []

for df_chunk in tqdm(pd.read_csv(path,usecols=cols, chunksize=chunksize,nrows=nrows)):
    df_list.append(df_chunk)
    
# Merge all dataframes into one dataframe
data = pd.concat(df_list)
# # Delete the dataframe list to release memory
# del df_list, df_chunk

5500it [01:50, 49.57it/s]


In [3]:
%%time
print(data[::10])

          fare_amount          pickup_datetime  pickup_longitude  \
0                 4.5  2009-06-15 17:26:21 UTC        -73.844311   
10                5.3  2012-04-08 07:30:50 UTC        -73.996335   
20               10.5  2010-09-07 13:18:00 UTC        -73.985382   
30                5.5  2014-02-19 16:03:00 UTC        -73.976075   
40                8.1  2011-03-29 15:39:49 UTC        -73.984267   
...               ...                      ...               ...   
54999950         11.7  2009-03-01 14:04:00 UTC        -73.952725   
54999960         18.9  2012-02-01 07:42:00 UTC        -74.017390   
54999970          6.5  2012-10-13 09:54:00 UTC        -74.001070   
54999980         13.7  2010-09-03 17:28:20 UTC        -73.951656   
54999990         56.3  2011-07-26 06:27:00 UTC        -73.983975   

          pickup_latitude  dropoff_longitude  dropoff_latitude  \
0               40.721319         -73.841610         40.712278   
10              40.737142         -73.980721       

### Let's take a look at the data

In [4]:
%%time
data.describe()

CPU times: user 19.8 s, sys: 18 s, total: 37.8 s
Wall time: 41.7 s


,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,5.500000e+07,5.500000e+07,5.500000e+07,5.499962e+07,5.499962e+07,5.500000e+07
mean,1.134515e+01,-7.250967e+01,3.991983e+01,-7.251120e+01,3.992063e+01,1.685380e+00
std,2.077308e+01,1.285500e+01,9.647758e+00,1.278611e+01,9.622323e+00,1.327813e+00
min,-3.000000e+02,-3.442060e+03,-3.492264e+03,-3.442025e+03,-3.547887e+03,0.000000e+00
25%,6.000000e+00,-7.399207e+01,4.073493e+01,-7.399140e+01,4.073403e+01,1.000000e+00
50%,8.500000e+00,-7.398180e+01,4.075265e+01,-7.398015e+01,4.075316e+01,1.000000e+00
75%,1.250000e+01,-7.396708e+01,4.076713e+01,-7.396367e+01,4.076810e+01,2.000000e+00
max,9.396336e+04,3.457626e+03,3.408790e+03,3.457622e+03,3.537133e+03,2.080000e+02


###  Check for missing values


In [5]:
%%time
# Checking for missing values
data.isnull().sum()

CPU times: user 6.13 s, sys: 14 s, total: 20.1 s
Wall time: 28 s


fare_amount            0
pickup_datetime        0
pickup_longitude       0
pickup_latitude        0
dropoff_longitude    375
dropoff_latitude     375
passenger_count        0
dtype: int64

### From our observation, there are some missing values, so we replace it with the mean of that column

In [6]:
%%time
# Replace missing fields with the mean of that column
# data.fillna(data.mean(), inplace=True)
# replacing NA values in "dropoff_longitude" column with it's mean
data['dropoff_longitude'].fillna(data['dropoff_longitude'].mean(), inplace = True)

CPU times: user 257 ms, sys: 569 ms, total: 826 ms
Wall time: 874 ms


In [7]:
%%time
# replacing NA values in "dropoff_latitude" column with it's mean
data['dropoff_latitude'].fillna(data['dropoff_latitude'].mean(), inplace = True)

CPU times: user 247 ms, sys: 620 ms, total: 867 ms
Wall time: 1.05 s


###  From the description above, some of the fare_amount have negative values

In [8]:
%%time
data_negative_fare_amount = data[(data['fare_amount'] < 0)]
data_negative_fare_amount.shape
print(f'Number of records with negative fare_amount: {data_negative_fare_amount.shape[0]}')

Number of records with negative fare_amount: 2437
CPU times: user 328 ms, sys: 1.73 s, total: 2.06 s
Wall time: 3.28 s


#### We are going to remove all negatvie fare_amounts

In [9]:
%%time
data = data[(data['fare_amount'] > 0)]


CPU times: user 6.23 s, sys: 22.2 s, total: 28.4 s
Wall time: 42.9 s


In [10]:
%%time
data.describe()

CPU times: user 15.4 s, sys: 15.6 s, total: 30.9 s
Wall time: 39.7 s


,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,5.499620e+07,5.499620e+07,5.499620e+07,5.499620e+07,5.499620e+07,5.499620e+07
mean,1.134638e+01,-7.250996e+01,3.991999e+01,-7.251169e+01,3.992089e+01,1.685376e+00
std,2.077289e+01,1.285436e+01,9.647652e+00,1.278486e+01,9.621960e+00,1.327812e+00
min,1.000000e-02,-3.442060e+03,-3.492264e+03,-3.442025e+03,-3.547887e+03,0.000000e+00
25%,6.000000e+00,-7.399207e+01,4.073493e+01,-7.399140e+01,4.073403e+01,1.000000e+00
50%,8.500000e+00,-7.398180e+01,4.075265e+01,-7.398015e+01,4.075316e+01,1.000000e+00
75%,1.250000e+01,-7.396708e+01,4.076713e+01,-7.396367e+01,4.076810e+01,2.000000e+00
max,9.396336e+04,3.457626e+03,3.408790e+03,3.457622e+03,3.537133e+03,2.080000e+02


### In further observation, we have pickup_longitude, puckup_latitiude, dropoff_longitude and dropoff_latitude. We can calculate the distance

In [11]:
# Function that calculates distance between pickup location and dropoff location
def getDistance(lat1,lon1,lat2,lon2):
    r = 6378 # earth's radius
    lat1 = np.deg2rad(lat1)
    lon1 = np.deg2rad(lon1)
    lat2 = np.deg2rad(lat2)
    lon2 = np.deg2rad(lon2)
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = r*c
    
    return distance

In [13]:
%%time
# Add new cloumn "distance" to the data
data['distance'] = getDistance(data.pickup_latitude, data.pickup_longitude, 
                                      data.dropoff_latitude, data.dropoff_longitude)

CPU times: user 16.7 s, sys: 11.5 s, total: 28.2 s
Wall time: 20.9 s


In [14]:
%%time
data.head()

CPU times: user 2.04 ms, sys: 4.02 ms, total: 6.06 ms
Wall time: 7.18 ms


,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance
0,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1,1.031896
1,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1,8.459418
2,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2,1.391052
3,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1,2.802346
4,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1,2.001353


In [ ]:
feature_cols = ['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count']

X = data[feature_cols] 
y = data['fare_amount']

In [ ]:
X = preprocessing.scale(X)
# i don't think we should scale y (target). Professor Mo said to only scale features columns
y = preprocessing.scale(y)

# Delete the dataframe list to release memory
del data

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, 
                                                 random_state=2)
#55millions rows it = iteration(nrows per it) x nrows = total rows

print(X.shape)
print(y.shape)

#preprocessing.LabelEncoder()-convert string or float values to 0 .. n classes.
lab_enc = preprocessing.LabelEncoder()
y_train_encoded = lab_enc.fit_transform(y_train)
y_encoded = lab_enc.fit_transform(y)

print(y[0:5])
print(y_encoded[0:5])

In [ ]:
#linear regression can also use y_train
my_linreg = LinearRegression()
my_linreg.fit(X_train, y_train_encoded)
y_prediction = my_linreg.predict(X_test)
mse = metrics.mean_squared_error(y_test, y_prediction)
rmse = np.sqrt(mse)
print(rmse)

In [ ]:
my_logreg = LogisticRegression(solver='liblinear', multi_class='auto')
my_logreg.fit(X_train, y_train_encoded)
y_prediction = my_logreg.predict(X_test)
mse = metrics.mean_squared_error(y_test, y_prediction)
rmse = np.sqrt(mse)
print(rmse)

In [ ]:
my_decisiontree = DecisionTreeClassifier(random_state=5)
my_decisiontree.fit(X_train, y_train_encoded)
y_prediction = my_decisiontree.predict(X_test)
mse = metrics.mean_squared_error(y_test, y_prediction)
rmse = np.sqrt(mse)
print(rmse)

Cross Validation

In [ ]:
score_list = cross_val_score(my_linreg, X, y_encoded, cv=10)
score = score_list.mean()
print('KNN: '+str(score))

In [ ]:
score_list = cross_val_score(my_logreg, X, y_encoded, cv=10)
score = score_list.mean()
print('Logistic Regression: '+str(score))

In [ ]:
score_list = cross_val_score(my_decisiontree, X, y_encoded, cv=10)
score = score_list.mean()
print('Decision Tree: '+str(score))